# PROYECTO DE INTELIGENCIA ARTIFICIAL: Seleccion de características

Las siguientes funciones se encargan de obtener los atributos con los que trabajaremos, ademas de obtener los atributos ordenados y los objetivos para ambos csv.

Importaciones:

In [4]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

from sklearn.tree import DecisionTreeClassifier

from matplotlib import pyplot
from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer

In [5]:
def lectura_titanic():
    fichero = pd.read_csv('titanic.csv')    
    return fichero

def lectura_breastCancer():
    fichero = pd.read_csv('BreastCancer.csv')
    return fichero

In [6]:
titanic = lectura_titanic()
breast_cancer = lectura_breastCancer()

In [7]:
# Eleccion de semilla inicial aleatoria.
np.random.seed(357823)

# Atributos discretos, continuos y objetivo de titanic.csv.
atributos_discretos_titanic = ['Sex', 'Embarked', 'Alone', 'Deck']
atributos_continuos_titanic = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Initial', 
                           'Age_band', 'Family_Size', 'Fare_cat', 'Title', 'Is_Married']
atributos_titanic = titanic.loc[:, atributos_discretos_titanic + atributos_continuos_titanic]
objetivo_titanic = titanic['Survived']

# Atributos discretos, continuos y objetivo de BreastCancer.csv.
atributos_continuos_breast_cancer = ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 
                           'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 
                           'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error',
                           'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 
                           'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points',
                           'worst symmetry', 'worst fractal dimension']
																				
atributos_breast_cancer = breast_cancer.loc[:, atributos_continuos_breast_cancer]
objetivo_breast_cancer = breast_cancer['diagnosis']

##### A continuación realizaremos los algoritmos de busqueda con los que vamos a trabajar. Para ello, trabajaremos con el algoritmo de "Decision tree clasifier" y el algoritmo de "Naives bayes".

#### Decision tree clasifier

In [25]:
def decision_tree(atributos, objetivo, atributos_discretos):
    codificador_atributos_discretos = OrdinalEncoder()
    codificador_atributos_discretos.fit(atributos[atributos_discretos])
    
    atributos[atributos_discretos] = codificador_atributos_discretos.transform(atributos[atributos_discretos])

    codificador_objetivo = LabelEncoder()
    objetivo = codificador_objetivo.fit_transform(objetivo)

    clasificador_CART = DecisionTreeClassifier()
    clasificador_CART.fit(atributos, objetivo)
    
    return clasificador_CART

In [26]:
# DECISION TREE CLASIFIER
decision_tree(atributos_titanic, objetivo_titanic, atributos_discretos_titanic)

DecisionTreeClassifier()

#### Funcion de rendimiento

In [27]:
def rendimiento(atributos, objetivo, feature, atributos_discretos):
    (atributos_entrenamiento, atributos_prueba,
     objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo)
    tree = decision_tree(atributos, objetivo, atributos_discretos)
    score = cross_validate(tree, atributos_entrenamiento[feature], objetivo_entrenamiento, cv = 7, scoring = 'balanced_accuracy')
    return np.mean(score['test_score'])

In [28]:
feature = ['Pclass', 'Age']
rendimiento(atributos_titanic, objetivo_titanic, feature, atributos_discretos_titanic)

0.6464891994845912

In [29]:
def busqueda_secuencial_hacia_atras(atributos, objetivo, atributos_discretos, n_exp=10, cv=7):
    # Inicialización
    solucion_actual = list(atributos.columns)
    n_features = len(solucion_actual)
    resultados = []
    
    # Ejecución
    while len(solucion_actual) > 0:
        rendimiento_actual = rendimiento(atributos, objetivo, solucion_actual, atributos_discretos)
        resultados.append((solucion_actual.copy(), len(solucion_actual), rendimiento_actual))
        
        if len(solucion_actual) == 1:
            break
        
        peor_rendimiento = float('inf')
        peor_feature = None
        
        # Evaluar todas las posibles soluciones temporales
        for feature in solucion_actual:
            solucion_temporal = solucion_actual.copy()
            solucion_temporal.remove(feature)
            rendimiento_temporal = rendimiento(atributos, objetivo, solucion_temporal, atributos_discretos)
            
            # Guardar el rendimiento de la peor característica
            if rendimiento_temporal < peor_rendimiento:
                peor_rendimiento = rendimiento_temporal
                peor_feature = feature
        
        # Actualizar la solución actual eliminando la peor característica
        solucion_actual.remove(peor_feature)
    
    # Crear un DataFrame con los resultados
    df_resultados = pd.DataFrame(resultados, columns=['MejorSolucionTemporal', 'Tamaño', 'Rendimiento']).sort_values(by='Tamaño')
    
    # Devolver los resultados en forma de DataFrame
    return df_resultados

# Ejemplo de uso
titanic = lectura_titanic()
atributos_discretos_titanic = ['Sex', 'Embarked', 'Alone', 'Deck']
atributos_continuos_titanic = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Initial', 
                               'Age_band', 'Family_Size', 'Fare_cat', 'Title', 'Is_Married']
atributos_titanic = titanic.loc[:, atributos_discretos_titanic + atributos_continuos_titanic]
objetivo_titanic = titanic['Survived']

# Codificación de atributos discretos y objetivo
codificador_atributos_discretos_titanic = OrdinalEncoder()
atributos_titanic[atributos_discretos_titanic] = codificador_atributos_discretos_titanic.fit_transform(
    atributos_titanic[atributos_discretos_titanic]
)

codificador_objetivo_titanic = LabelEncoder()
objetivo_titanic = codificador_objetivo_titanic.fit_transform(objetivo_titanic)

# Aplicar la búsqueda secuencial hacia atrás en los datos de entrenamiento
resultados = busqueda_secuencial_hacia_atras(atributos_entrenamiento, objetivo_entrenamiento, atributos_discretos_titanic)

# Mostrar los resultados en forma de tabla
print(resultados)    

                                MejorSolucionTemporal  Tamaño  Rendimiento
14                                         [Embarked]       1     0.579354
13                                  [Embarked, Alone]       2     0.535007
12                             [Embarked, Alone, Age]       3     0.582295
11                      [Embarked, Alone, Age, SibSp]       4     0.624031
10               [Embarked, Alone, Age, SibSp, Parch]       5     0.596148
9          [Embarked, Alone, Age, SibSp, Parch, Fare]       6     0.653750
8    [Embarked, Alone, Deck, Age, SibSp, Parch, Fare]       7     0.695786
7   [Embarked, Alone, Deck, Age, SibSp, Parch, Far...       8     0.674794
6   [Sex, Embarked, Alone, Deck, Age, SibSp, Parch...       9     0.743571
5   [Sex, Embarked, Alone, Deck, Age, SibSp, Parch...      10     0.773371
4   [Sex, Embarked, Alone, Deck, Age, SibSp, Parch...      11     0.727411
3   [Sex, Embarked, Alone, Deck, Age, SibSp, Parch...      12     0.757710
2   [Sex, Embarked, Alone

In [ ]:
def busqueda_secuencial_hacia_atras_mixta(atributos, objetivo, atributos_discretos, iteraciones=10, n_exp=10, cv=7):
    # Inicialización
    solucion_actual = list(atributos.columns)
    n_features = len(solucion_actual)
    añadidos = []
    eliminados = []
    resultados = []
    contador = 0

     while len(solucion_actual) > 0:
        rendimiento_actual = rendimiento(atributos, objetivo, solucion_actual, atributos_discretos)
        resultados.append((solucion_actual.copy(), len(solucion_actual), rendimiento_actual))
        
        if len(solucion_actual) == 1:
            break
        
        peor_rendimiento = float('inf')
        peor_feature = None
        
        # Evaluar todas las posibles soluciones temporales
        for feature in solucion_actual:
            solucion_temporal = solucion_actual.copy()
            solucion_temporal.remove(feature)
            rendimiento_temporal = rendimiento(atributos, objetivo, solucion_temporal, atributos_discretos)
            
            # Guardar el rendimiento de la peor característica
            if rendimiento_temporal < peor_rendimiento:
                peor_rendimiento = rendimiento_temporal
                peor_feature = feature
        
        # Actualizar eliminados añadiendo la peor caracteristica y la solución actual eliminando la peor característica
        eliminados.append(peor_feature)
        solucion_actual.remove(peor_feature)

        

    






